In [2]:
import numpy as np
import pytreenet as ptn
from copy import deepcopy
from scipy.linalg import expm

In [ ]:
"""
First did it like this, but I guess completely_contract_tree is more 
costly that moving the canoical center. I wonder why the results are
not same!
"""

def reduced_density_matrix(ttn: ptn.TreeTensorNetwork ,node_id : str) -> np.ndarray:
    """
    Computes the reduced density matrix of a node in a tree tensor network.
    Args :
        ttn: Tensor
        node_id: str
    Returns :
        pho : Tensor
    """
    pho , order = density_tensor(ttn)
    dim = pho.ndim
    for i in range(len(order)):
        if order[i] == node_id:
            count = i
    for i in range(count):
        pho = np.trace(pho, axis1 = 0, axis2 = pho.ndim//2)
    for i in range(count +1 , dim//2):
        pho = np.trace(pho, axis1 = 1, axis2 = 1 + pho.ndim//2)
    return pho

def density_tensor(ttn: ptn.TreeTensorNetwork) -> (np.ndarray, List[str]):
    """
    Computes the density tensor of a tree tensor network.
    Args :
        ttn: Tensor
    Returns :
        density tensor : Tensor , 
        order : List[str]
        # order of legs =  [out_1, out_2, ..., out_n, in_1, in_2, ..., in_n] 
    """    
    ttn = normalize_ttn(ttn)
    ttn_cct = deepcopy(ttn)
    tensor , order = ttn_cct.completely_contract_tree()

    ket = tensor
    bra = tensor.conj()
    ket = ket.reshape(ket.shape + (1,))
    bra = bra.reshape(bra.shape + (1,))

    return  np.tensordot(bra,ket,axes=([-1],[-1])) , order

""""
I wonder why the results are not same!
"""
ttn = ptn.random_big_ttns_two_root_children()
 # pho , order = density_tensor(ttn) 
# verryyy costly

In [1]:
def density_ttno(ttn):
    tensor , order = density_tensor(ttn)
    leg_dict = {order[i] : i for i in range(len(order))}
    return ptn.TTNO.from_tensor(ttn, tensor, leg_dict)

pho = density_ttno(mps)

NameError: name 'mps' is not defined

In [ ]:
def ttno_trace(pho):
    ttno_cct = deepcopy(pho)
    ttno_cct = ttno_cct.completely_contract_tree()[0] 
    for _ in range(ttno_cct.ndim//2):
        ttno_cct = np.trace(ttno_cct, axis1 = 0, axis2 = 1)
    return ttno_cct

ttno_trace(pho[0])

In [ ]:
from typing import List, Union, Any, Dict
from pytreenet import TDVPUpdatePathFinder
from pytreenet import compute_transfer_tensor
from pytreenet import SplitMode

def move_orth_for_path(ttn,path: List[str]):
        if len(path) == 0:
            return
        assert ttn.orthogonality_center_id == path[0]
        for i, node_id in enumerate(path[1:]):
            ttn.move_orthogonalization_center(node_id,mode=SplitMode.KEEP)
                        
def reduced_density(ttn): 
    update_path = TDVPUpdatePathFinder(ttn).find_path()
    ttn.canonical_form(update_path[0],mode=SplitMode.KEEP)

    orthogonalization_path = []
    for i in range(len(update_path)-1):
        sub_path = ttn.path_from_to(update_path[i], update_path[i+1])
        orthogonalization_path.append(sub_path[1::])

    dict = {}
    for i, node_id in enumerate(update_path):
        contracted_legs = tuple(range(ttn.tensors[node_id].ndim - 1 )) 
        if i == len(update_path)-1:
            reduced_density = compute_transfer_tensor(ttn.tensors[node_id], contracted_legs)        
            dict[node_id] = reduced_density
        elif i == 0:
            reduced_density = compute_transfer_tensor(ttn.tensors[node_id], contracted_legs)        
            dict[node_id] = reduced_density
            next_node_id = orthogonalization_path[0][0]
            move_orth_for_path(ttn,[node_id, next_node_id])
        else:
            current_orth_path = orthogonalization_path[i-1]
            move_orth_for_path(ttn,current_orth_path)
            reduced_density = compute_transfer_tensor(ttn.tensors[node_id], contracted_legs)        
            dict[node_id] = reduced_density
            next_node_id = orthogonalization_path[i][0]
            move_orth_for_path(ttn,[node_id, next_node_id])
    return dict        

In [ ]:
def orthogonalize_against(ttn1 , ttn2):
    for i in range(len(ttn1.nodes)):
        a = ttn1.tensors[f"site{i}"]
        b = deepcopy(ttn2.tensors[f"site{i}"])
        b = normalize_tensor(b)
        indices = tuple(range(b.ndim))
        prod = np.tensordot(b.conj(), a , axes = (indices,indices) )
        a = a - b * prod
        a = a / np.sqrt(ptn.contract_two_ttns(ttn1,ttn1.conjugate()))
        ttn1.tensors[f"site{i}"] = a
        

In [ ]:
a = ptn.crandn((5,2))
b = ptn.crandn((5,2))
b = normalize_tensor(b)
indices = tuple(range(b.ndim))
prod = np.tensordot(b.conj(),a , axes = (indices,indices) )
a = a - b * prod
a = a / np.sqrt(np.tensordot(a,a.conj(), axes = (indices , indices) ))

In [ ]:
np.tensordot(a,a.conj() , axes = (indices,indices) )

In [ ]:
np.tensordot(b,b.conj() , axes = (indices,indices) )

In [ ]:
np.tensordot(a,b.conj() , axes = (indices,indices) )

In [ ]:
i=1
np.tensordot(mps1.tensors[f"site{i}"],mps2.tensors[f"site{i}"],axes=((0,1,2),(0,1,2)))

In [ ]:
mps1 = ptn.MatrixProductState.from_tensor_list(tensors1,root_site=5,node_prefix="site")
ptn.canonical_form2(mps1)
mps1 = normalize(mps1)
mps2 = ptn.MatrixProductState.from_tensor_list(tensors2,root_site=5,node_prefix="site")
ptn.canonical_form2(mps2)
mps2 = normalize(mps2)
orthogonalize_against(mps2 , mps1)
ptn.contract_two_ttns(mps2, mps1.conjugate())


In [ ]:
np.tensordot(mps1.tensors["site1"], mps2.tensors["site1"] ,axes = ((0,1,2),(0,1,2)))

In [ ]:
np.tensordot(mps1.tensors["site2"],mps2.tensors["site2"].conj(),axes = ((0,1,2),(0,1,2)))

In [ ]:
shapes = [(5, 2), (5, 7, 2), (7, 3, 2), (3, 6, 2), (6, 30, 2), (30, 2)]
tensors1 = [ptn.crandn(shape) for shape in shapes]
tensors2 = [ptn.crandn(shape) for shape in shapes]
tensors3 = [ptn.crandn(shape) for shape in shapes]
tensors4 = [ptn.crandn(shape) for shape in shapes]

mps1 = ptn.MatrixProductState.from_tensor_list(tensors1,root_site=5,node_prefix="site")
tp = ptn.random_tensor_product(mps1, num_operators= len(mps1))
tensor = ptn.crandn([2,2,2,2,2,2,
                     2,2,2,2,2,2])
leg_dict = {"site0": 0, "site1": 1, "site2": 2, "site3": 3, "site4": 4, "site5": 5}
hamiltonian = ptn.TTNO.from_tensor(mps1, tensor, leg_dict)

results = Krylov_space(mps1,hamiltonian,2)

# ptn.canonical_form2(mps1)
mps1 = normalize(mps1)

mps2 = results[1]
# ptn.canonical_form2(mps2)
mps2 = normalize(mps2)
# orthogonalize_against(mps2 , mps1)

mps3 = results[2]
# ptn.canonical_form2(mps3)
mps3 = normalize(mps3)
# orthogonalize_against(mps3 , mps1)

In [ ]:
mps1.nodes["site5"]

In [ ]:
ptn.contract_two_ttns(mps1, mps1.conjugate())

In [ ]:
U , S , B = ptn.truncated_tensor_svd(mps1.tensors["site4"], (0,2), (1,),
                                     max_bond_dim=np.inf , rel_tol=-np.inf, total_tol=-np.inf)
print(U.shape , S.shape , B.shape)


In [ ]:
B_dagger = B.conj().T
B_dagger_B = B_dagger @ B 
P = np.eye(B_dagger_B.shape[0]) - B_dagger_B

In [ ]:
pho.shape

In [ ]:
pho_2 = reduced_density_matrix(mps2,"site4")
pho_3 = reduced_density_matrix(mps3,"site4")
pho_4 = reduced_density_matrix(mps4,"site4")
pho = pho_2 + pho_3 + pho_4

pho_baar = P @ pho @ P

In [ ]:
A = np.array([[13, 5],
              [5, 7]])

w, v = np.linalg.eig(A)
B_bar = np.conjugate(v.T)
S = np.diag(np.sqrt(w))  

# Reconstruct A from B and S
np.allclose( A , np.dot(np.conjugate(B_bar.T), np.dot(S@S, B_bar)) )